# Simple Bigram Language Model
Following the example by Andrej Karpathy, we build a simple character-level language model to **generate** text. The model is a single layer NN that recieves a single token/character as input and then predicts the next token/character.

The data used for training this generative model is ["Tiny Shakespeare"](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt), which is a concatenation of all of the works of Shakespear's (in a single file) -- about 1MB of data.

### Key concepts:
1. Bigram: a sequence of two adjacent elements in a sequence of tokens, such as words, letters, or other units of text. 
2. batch_size: How many independent sequences will be processed in parallel.
3. block_size: max sequence length (aka context) that will be used for prediction.

## The plan
1. Load and inspect the data
2. Define the vocabulary, which is the total number of unique tokens in a given corpus. Here, we tokenize at the character level, where each character in a piece of text is a separate token. We end up with $65$ tokens.
3. Create mappings from tokens to indices (encode) and visa versa (decode).
4. Split data into train / validate sets. Given we are building a generative model that uses previous text to generate the next set of tokens, we can't assume the tokens are i.i.d., so we don't shuffle before the split.
6. Define a `BigramLM` model to learn the token embeddings matrix (which is actually a weights matrix). After training, the weights can be used to predict the next token/character, and generate text sequences. Here, the weights (embeddings matrix elements) are in fact the bigram probabilities, so the embeddings matrix is of shape $65 \times 65$.
7. Inspect some generated text output(s) before training the model (from randomly initiated weights).
8. Train the model. Generate text giving a single token input. Inspect the generated text after training.

## Results
After training, the generated sequences begin to have some of the characteristics of the Shakespearian text -- we observe words and paragraphs. So training definitely improved predictions, but it has a ways to go.


In [1]:
import numpy as np
import torch
from torch.nn import functional as F
torch.manual_seed(12345)

In [2]:
with open("input.txt", "r", encoding='utf-8') as f:
    text = f.read()

In [3]:
# inspect first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# get sorted unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print(f"\nVocab size is: {vocab_size}")

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

Vocab size is: 65


In [5]:
# create mappings from characters to integers and visa versa
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

assert stoi["!"] == 2
assert stoi["z"] == 64
assert itos[2] == "!"
assert itos[64] == "z"

In [6]:
# turn a string into a list of integers
encode = lambda s: [stoi[c] for c in s]
# turn a list of integers into a string
decode = lambda L: "".join([itos[i] for i in L])

print(f"encode('Welcome home!') --> {encode('Welcome home!')}")
print(f"decode(encode('Welcome home!')) --> {decode(encode('Welcome home!'))}")


encode('Welcome home!') --> [35, 43, 50, 41, 53, 51, 43, 1, 46, 53, 51, 43, 2]
decode(encode('Welcome home!')) --> Welcome home!


In [7]:
# encode the entire text and store in a torch tensor
data = torch.tensor(encode(text))
print(data.shape, data.type)

torch.Size([1115394]) <built-in method type of Tensor object at 0x1391fb4f0>


In [8]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# decode requires a list input
print(decode(data[:100].tolist()))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [10]:
# split data into train / validate sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n+1:]

In [11]:
# set blocksize
torch.manual_seed(1337)
block_size = 8
print(data[:block_size+1])
print(decode(data[:block_size+1].tolist()))


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
First Cit


In [12]:
# For illustration only:
# auto-regressive behavior where every previous token in a block is part of the input
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"context: {context}, target: {target}")

# However, that's not what we work with in this notebook

context: tensor([18]), target: 47
context: tensor([18, 47]), target: 56
context: tensor([18, 47, 56]), target: 57
context: tensor([18, 47, 56, 57]), target: 58
context: tensor([18, 47, 56, 57, 58]), target: 1
context: tensor([18, 47, 56, 57, 58,  1]), target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


In [13]:
torch.manual_seed(1337)
batch_size=4
block_size=8
# focus on input-output where the input is the previous token
def get_batch(split):
    """Generate a small batch of data (inputs x and associated targets y)
    from the train set or validation set.

    Parameters:
        split (str): A string indicating whether to use 'train' or 'validate' data split.

    Returns:
        tuple: A tuple containing two torch tensors:
            - x (torch.Tensor): A tensor containing input data.
            - y (torch.Tensor): A tensor containing target data.
    """
    df = train_data if "train" else val_data
    # randomly pick the start of the batch_size (e.g. 4) batches in the data
    ix = torch.randint(0, len(df) - block_size, (batch_size,))
    x = torch.stack([df[i:i+block_size] for i in ix])
    y = torch.stack([df[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print(xb)
print(yb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


## Define BigramLM model
`BigramLM` is a character-level language model that primarily relies on an embedding layer as its input layer and computes logits as part of its output. Its primary purpose is to learn and estimate bigram transition probabilities between characters in text data and generate text based on those probabilities.

In [14]:
# Example to illustrate torch.nn.Embedding

# The list of tokens
tokens = torch.tensor([0,5,9], dtype=torch.long)
# Define an embedding layer, where you know upfront that in total you
# have 10 distinct words, and you want each word to be encoded with
# a 20 dimensional vector
embedding = torch.nn.Embedding(num_embeddings=10, embedding_dim=20)
# Obtain the embeddings for each of the words in the sentence
embedded_output = embedding(tokens)
print(tokens)
print(embedded_output)

tensor([0, 5, 9])
tensor([[ 0.6258,  0.0255,  0.9545,  0.0643, -0.5024, -0.2026, -1.5671, -1.0980,
          0.2360, -0.2398, -0.9211,  1.5433, -0.3676, -0.7483, -0.1006,  0.7307,
         -2.0371,  0.4931,  1.4870,  0.5910],
        [ 0.3404,  1.1685, -0.6526,  0.3768,  0.1209,  2.5418, -0.6405, -1.9740,
         -1.1572,  0.2896,  0.6164, -0.4370,  0.1670,  0.4586, -1.7662,  0.5860,
          0.5873,  0.2861,  0.0083, -0.2523],
        [ 0.8475,  0.0774,  0.5433, -0.8438, -0.7864,  0.2444, -0.9812, -0.0699,
          0.2984, -0.7264, -0.3119, -0.4560,  1.8354,  1.4473, -0.7374,  0.2485,
          0.5042,  0.8713, -0.2742, -0.7469]], grad_fn=<EmbeddingBackward0>)


In [15]:
class BigramLM(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = torch.nn.Embedding(num_embeddings=vocab_size,
                                                        embedding_dim=vocab_size)

    def forward(self, idx, y=None):
        # both idx and y are of size batch_size x block size (aka B x T)
        # logits is of size (B x T x C), where C is number of classes
        logits = self.token_embedding_table(idx)
        B, T, C = logits.shape 
        if y == None:
            loss = None
        else:
            # torch.nn.functional.cross_entropy requires size (N, C)
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits, y)
        return logits, loss

    # will use a model to generate in the general case when we use
    # more previous tokens to generate the next; but in this example
    # we're obviously using only a single token to generate the next
    def generate(self, idx, max_new_tokens):
        '''Take the idx sequence which is (B, T) and extend it
        sequentially in the time dimention to (B, T+1), ..., (B, T+max_new_tokens)
        '''
        for _ in range(max_new_tokens):
            # idx is (B, T) array of indices 
            logits, _ = self(idx)
            # forcus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # In theory, the sum of class probabilities should be one for each batch
            # However, multinomial does not require probs to sum to one (in which case it uses the values as weights)
            # Generate next token:
            # don't simply output the class with highest probability; generate from a multinomial distribution
            next_idx = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, next_idx), dim=1)
        return idx
        

In [16]:
# inspect output before training.
model = BigramLM(vocab_size)
logits, loss = model(xb, yb)
print(f"Logits shape: {logits.shape}") # (batch_size * block_size, vocab_size)
print(f"Loss before training: {loss.item()}")
# a single batch
kickoff_index = torch.zeros((1,1), dtype=torch.long)
out = model.generate(kickoff_index, max_new_tokens=100)[0]
print("A generated sequence:")
print(decode(out.tolist()))

Logits shape: torch.Size([32, 65])
Loss before training: 4.360136985778809
A generated sequence:

cwgn,qwYfgBWd'CLI TNla-YQQCfm-nBerjt:.djA;.Q!X&i$g Pvlb BSvfBf
krTX$d?XazHws-lo?-aqKyGOZX;rYi,qf$dDO


In [17]:
# generate a batch of 4 sequences starting with the same toekn, each with max_new_tokens=100
# notice that we get 4 different results (because we 'generate' from a distribution)
kickoff_index = torch.zeros((4,1), dtype=torch.long)
out = model.generate(kickoff_index, max_new_tokens=100)
print("Generated sequences:")
print(decode(out[0].tolist()))
print(decode(out[1].tolist()))
print(decode(out[2].tolist()))
print(decode(out[3].tolist()))

Generated sequences:

;L'yvCf$Bn.BAmoscV&W;PX pQ:LnHNt!yKOA;-,,,TFJXpJ
aIv rrKE!uXpFP3CENq
tLu?l wUnjxerlRBdYYK -Pb$K,3HsE

Vd!i,DNUlt:B&NLR!tLhP$x:RV&jErZH?jEpS&WZSKplQCOIU.Ine!!!MgBqL
AJmR:-Ikh:;D-nczTfQEf'UcBdTfAeqK ZX;b 

m&bQLsAx:dtm'rkjnae
.IfQrg kC-d,kJj:okq;tjECwv ZHxHT!lyvKETi,;,LQY';c;oMT.B .:vzxkgugE:WIlhLZmw:pPln

3arrgOEIvKyvhrkZWH.BRarrVR ZsWd;bdfapSuu-j
gVx VJrAvf
utGEYzd'lvxsUBWdJtsDiZYBoerrlsD'lNtmvHVzE;GHpH


In [18]:
# train the model
model = BigramLM(vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
batch_size = 32
for epoch in range(10000):
    optimizer.zero_grad(set_to_none=True)
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate loss
    logits, loss = model(xb, yb)
    loss.backward()
    optimizer.step()
    if epoch % 500 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")


Epoch: 0, Loss: 4.5606770515441895
Epoch: 500, Loss: 4.118853569030762
Epoch: 1000, Loss: 3.6763458251953125
Epoch: 1500, Loss: 3.3370871543884277
Epoch: 2000, Loss: 3.0861566066741943
Epoch: 2500, Loss: 2.9228947162628174
Epoch: 3000, Loss: 2.79687237739563
Epoch: 3500, Loss: 2.5935275554656982
Epoch: 4000, Loss: 2.5961170196533203
Epoch: 4500, Loss: 2.612950563430786
Epoch: 5000, Loss: 2.553447723388672
Epoch: 5500, Loss: 2.478346347808838
Epoch: 6000, Loss: 2.410762310028076
Epoch: 6500, Loss: 2.5325160026550293
Epoch: 7000, Loss: 2.498384952545166
Epoch: 7500, Loss: 2.523305654525757
Epoch: 8000, Loss: 2.537670135498047
Epoch: 8500, Loss: 2.3776967525482178
Epoch: 9000, Loss: 2.3792474269866943
Epoch: 9500, Loss: 2.4181370735168457


In [19]:
# generate 4 sequences from trained model
generated_idx = model.generate(kickoff_index, max_new_tokens=300)
print(decode(generated_idx[0].tolist()))


Angee dal whayo l, chDoure han lfr t cif y ter hitsh!
N irir theenofo's d k thapa, e eow heane loun wise mig t nofowaginthe nd t owixphenggherun Hathe.
LUSlows, noterd Whatu y my weller Y dld hed o's:
ENGoworru,
Cat; kesherew'YCllllleareMERKICHqund ithar'liconfrn, me NCLieas ixtoukss ug; l.
O duragr


In [20]:
print(decode(generated_idx[1].tolist()), '\n')


Murysurm, wifa ick bsoothe,
GBwerdwirun,
bu IZZULit, n'd pend an by t pr ingand?
Ber'dy th bl m at he t th sthind t loburisthard I thene bl omes mpid st. bichthare w w, fany IOLELOUSerecheld, alir's? aty pitenf wst htrord,
GLO: cbrise anvethithiencorthaplanp, derhes me s yallofan'eleayond,
Towor bal 



In [21]:
print(decode(generated_idx[2].tolist()), '\n')



DUCind Cofatole youblird-ctontincr y!
HA.

Doud
IIOn m; untceen aninhis shy pe'sshen ise he y mis at warmsleeabed, od,
S s f a g ge yzeserilathouc-hour kishe 's, selathey h IIURIUShomiorcolpeane a fu avistisotocathe l,

Y my;-
Jap BWiven arous ghenevet vesue ind w shethos.
CLUCl.
I for dinet t be k 



In [22]:
print(decode(generated_idx[3].tolist()), '\n')


IZAnvent I f thunoce hethay he stesveating s ma ng jathad
S:
BEEOR:
Wrth me bl toucenombetheing situr turesuchian t
TO: hisgalotind coullle,
Then fofue, latrathowe bd POre ber awinRITEd dg oupithe itthennee be bu par touimen,
Whinthaye,
TI mice. l plaMO:
Tine u er touste le ayoffut gon, pllay fef nd 

